In [1]:
import cobra
import os

from utils.data_io import load_arabidopsis_model, get_proteome
from utils.genome_mapping import build_diamond_database, run_diamond, parse_diamond_output
from utils.model_building import (get_model_gene_to_proteome_map, 
                                  get_model_genes_to_remove, 
                                  prune_genes_to_remove, 
                                  generate_base_model, 
                                  gapfill_base_model)

# Build metabolic model

In this example notebook, we will build a sketch metabolic model for two Cotton plants, drawing our inspiration from the *Arabidopsis thaliana* model.

The notebook is built in such a way that files are re-downloaded every time. This ensures that any user is capable of running it, without having to upload heavy files to Github.

## 1. Load proteomes

First, we load the proteomes of our organisms of interest:

1. *Arabidopsis thaliana*: GCF_000001735.4
2. *Gossypium hirsutum*: GCF_007990345.1
3. *Gossypium arboreum*: GCF_000612285.1

In [2]:
# Get the Arabidopsis thaliana proteome
url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.4_TAIR10.1/GCF_000001735.4_TAIR10.1_translated_cds.faa.gz"
directory ='./genomes'
filename ="GCF_000001735.4_TAIR10.1_translated_cds.faa.gz"
get_proteome(url, directory, filename)


Directory './genomes' already exists.
File downloaded successfully: ./genomes/GCF_000001735.4_TAIR10.1_translated_cds.faa.gz


In [3]:
# Get the Cotton plants proteomes
# Gossypium hirsutum
url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/007/990/345/GCF_007990345.1_Gossypium_hirsutum_v2.1/GCF_007990345.1_Gossypium_hirsutum_v2.1_translated_cds.faa.gz"
directory ='./genomes'
filename ="GCF_007990345.1_Gossypium_hirsutum_v2.1_translated_cds.faa.gz"
get_proteome(url, directory, filename)

# Gossypium arboreum
url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/612/285/GCF_000612285.1_Gossypium_arboreum_v1.0/GCF_000612285.1_Gossypium_arboreum_v1.0_translated_cds.faa.gz"
directory ='./genomes'
filename ="GCF_000612285.1_Gossypium_arboreum_v1.0_translated_cds.faa.gz"
get_proteome(url, directory, filename)

Directory './genomes' already exists.
File downloaded successfully: ./genomes/GCF_007990345.1_Gossypium_hirsutum_v2.1_translated_cds.faa.gz
Directory './genomes' already exists.
File downloaded successfully: ./genomes/GCF_000612285.1_Gossypium_arboreum_v1.0_translated_cds.faa.gz


## 2. Run diamond

Running Diamond BlastP will allow mapping between organism's proteomes.

In [4]:
# Define the set of inputs that will be used to run the diamond BLASTP and parse the output
arabidopsis_proteome = "GCF_000001735.4_TAIR10.1_translated_cds.faa"
inputfile = "GCF_007990345.1_Gossypium_hirsutum_v2.1_translated_cds.faa"
database = "GCF_000001735.4_TAIR10.1_translated_cds.faa.dmnd"
outputfile = "A_thaliana_vs_G_hirsutum.tsv"
genome_path = "./genomes/"
diamond_path = "./genomes/diamond_output/"

In [5]:
# Goal is to get the list of model genes in each Cotton plant species
# We have to create a Diamond BLAST db for Arabidopsis thaliana
build_diamond_database(arabidopsis_proteome, genome_path=genome_path, outdir=diamond_path)

diamond v2.1.9.163 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: ./genomes/GCF_000001735.4_TAIR10.1_translated_cds.faa
Opening the database file...  [0.008s]
Loading sequences...  [0.194s]
Masking sequences...  [0.496s]
Writing sequences...  [0.047s]
Hashing sequences...  [0.016s]
Loading sequences...  [0s]
Writing trailer...  [0s]
Closing the input file...  [0s]
Closing the database file...  [0.011s]

Database sequences  48265
  Database letters  20841961
     Database hash  08f37904b6839a19ca64db937a97f479
        Total time  0.776000s


In [6]:
# Run Diamond BLASTP to identify proteins in Gossypium hirsutum that map to Arabidopsis thaliana
run_diamond(inputfile, database, outputfile, genome_path, diamond_path, input_type='protein')

In [7]:
# Get the output of Diamond that maps between the two species
diamond_mapping = parse_diamond_output(diamond_path, outputfile)
diamond_mapping.head(5)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,lcl|NC_053424.1_prot_XP_040972552.1_1,lcl|NC_003076.8_prot_NP_200440.1_45945,55.0,120,51,2,211,328,276,394,1.280000e-29,120.0
1,lcl|NC_053424.1_prot_XP_040972554.1_2,lcl|NC_003076.8_prot_NP_200440.1_45945,55.0,120,51,2,211,328,276,394,1.280000e-29,120.0
2,lcl|NC_053424.1_prot_XP_040972555.1_3,lcl|NC_003076.8_prot_NP_200440.1_45945,55.0,120,51,2,211,328,276,394,1.280000e-29,120.0
3,lcl|NC_053424.1_prot_XP_040972557.1_4,lcl|NC_003076.8_prot_NP_200440.1_45945,55.0,120,51,2,211,328,276,394,1.280000e-29,120.0
4,lcl|NC_053424.1_prot_XP_040972559.1_5,lcl|NC_003076.8_prot_NP_200440.1_45945,55.0,120,51,2,211,328,276,394,1.280000e-29,120.0


## 3. Build a metabolic model

1. Remove the genes and reactions from the Arabidopsis metabolic model to create a base model
2. Add the remaining genes and associated reactions by querying KEGG

In [8]:
# Get the Arabidopsis model
model = load_arabidopsis_model()
model.genes[0]

Gene identifier,ARTHCP007
Name,ARTHCP007
Memory address,0x7fb6f5472450
Functional,True
In 1 reaction(s),R00086_c


In [9]:
# Get the objective function, solution, uptake and secretion fluxes
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
S_CO2_c,Ex1,123,0,0.00%
S_Hydrogen_32_sulfide_c,Ex11,0.00286,0,0.00%
S_hv_p,Ex16,1000,0,0.00%
S_Orthophosphate_c,Ex18,0.001639,0,0.00%
S_H2O_c,Ex2,475.7,0,0.00%
S_Nitrate_c,Ex4,0.00814,0,0.00%
S_NH3_c,Ex5,0.2415,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
S_Oxygen_c,Ex3,-124,0,0.00%
S_Riboflavin_c,V0002,-0.00044,0,0.00%


### 3.1 Create a base model

In this model, the gene identifiers are consistent with KEGG entries: https://www.genome.jp/dbget-bin/www_bget?ath:ArthCp007

We thus need to first map the KEGG identifiers to the *A. thaliana* proteome used with Diamond in order to identify the genes to remove to create a base model.

In [10]:
# Map the Arabidopsis thaliana model genes to protein identifiers from the proteome
# This step will add the mapping directly to the diamond_mapping dataframe
diamond_mapping = get_model_gene_to_proteome_map(os.path.join(genome_path,arabidopsis_proteome), diamond_mapping)

In [11]:
# Get the genes to remove from the model
genes_to_remove = get_model_genes_to_remove(diamond_mapping, model)

14189 target species genes were found in the reference proteome
976 target species were found in the reference model containing 1404
428 genes must be removed from the reference model


In [12]:
# Make sure we are not removing genes that are essential individually or in combination
genes_to_remove = prune_genes_to_remove(model, genes_to_remove)
len(genes_to_remove)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
/home/jclachance/.pyenv/versions/3.11.4/envs/build_model_env/lib/python3.11/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]


415

In [13]:
# The base model is the model without the genes to remove
generate_base_model(genes_to_remove, os.path.join('./models/', 'hirsutum_base_model.json'))

In [14]:
# Verify if the base model can solve
base_model = cobra.io.load_json_model(os.path.join('./models','hirsutum_base_model.json'))
solution = base_model.optimize()
if solution.status == 'infeasible':
    print("The base model solves the model")
elif solution.status == 'optimal':
    print("The base model does not solve, you will need to run gapfilling")

The base model solves the model


/home/jclachance/.pyenv/versions/3.11.4/envs/build_model_env/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [16]:
# Gapfill the base model which cannot solve
gapfill_base_model(os.path.join('./models', 'hirsutum_base_model.json'), os.path.join('./models','hirsutum_functional_model.json'))

Loaded both models, now checking reactions one at a time


/home/jclachance/.pyenv/versions/3.11.4/envs/build_model_env/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Added R03509_c and it solves the model with value 10.0


### 3.2 Expand base model with organism specific genes and reactions

Thus far, we have built a skeleton metabolic model from a functional *Arabidopsis thaliana* reference model. The model was more than the sum of a set of reactions from KEGG. It contains a biomass objective function, a medium and is scaled to produce somewhat realistic predictions. We've thus used this template and reduced it such that it contains only *G. hirsutum* homologs. When we removed the genes that were not homologous, we ended up with a non-functional model: it could not produce biomass. This was somewhat expected, we blindly removed 1/3 of the genes and expected a complex system to just work? So we had to be careful, we excluded the individual essential genes and synthetic lethals from this list of genes to remove. 

Since our model couldn't solve, we had to patch this error. We took the easy way and identified a metabolic reaction from the *Arabidopsis thaliana* model that allowed the model to solve again.

The real (and hard) way, is to add back the *G. hirsutum* genes into this draft model. We could go there, but it would take **a lot of time**. Therefore, we only show what the first steps in this process might look like. We would identify:

1. Metabolic reactions from a public database and standardize their metabolite identifiers to those in the model (vice-versa, everyone should speak the smae language)
2. Identify the reactions that are not currently present in the model and add them back (including the gene reaction rule)
3. Test the ensuing phenotypes (we have to have some benchmark that says the model actually performs like a cotton plant cell does)

This process can be tricky and involves some manual curation, which is out of scope for this task.

In [17]:
# Load the base model
functional_model = cobra.io.load_json_model(os.path.join('./models','hirsutum_functional_model.json'))

In [20]:
import re
import requests
# Get the organism pathways from KEGG
response = requests.get(f"https://rest.kegg.jp/list/pathway/ghi")
print(response.status_code)
for line in response.text.split('\n'):
    print(line.split('\t'    ))

200
['ghi01100', 'Metabolic pathways - Gossypium hirsutum (upland cotton)']
['ghi01110', 'Biosynthesis of secondary metabolites - Gossypium hirsutum (upland cotton)']
['ghi01200', 'Carbon metabolism - Gossypium hirsutum (upland cotton)']
['ghi01210', '2-Oxocarboxylic acid metabolism - Gossypium hirsutum (upland cotton)']
['ghi01212', 'Fatty acid metabolism - Gossypium hirsutum (upland cotton)']
['ghi01230', 'Biosynthesis of amino acids - Gossypium hirsutum (upland cotton)']
['ghi01232', 'Nucleotide metabolism - Gossypium hirsutum (upland cotton)']
['ghi01250', 'Biosynthesis of nucleotide sugars - Gossypium hirsutum (upland cotton)']
['ghi01240', 'Biosynthesis of cofactors - Gossypium hirsutum (upland cotton)']
['ghi00010', 'Glycolysis / Gluconeogenesis - Gossypium hirsutum (upland cotton)']
['ghi00020', 'Citrate cycle (TCA cycle) - Gossypium hirsutum (upland cotton)']
['ghi00030', 'Pentose phosphate pathway - Gossypium hirsutum (upland cotton)']
['ghi00040', 'Pentose and glucuronate in

### Warning

The cell below takes ~15 minutes to run!

In [22]:
def query_kegg_ontology(ko_id):
    url = f"http://rest.kegg.jp/get/{ko_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f'Bad query {response.status_code}')

def get_kegg_reaction_id(content):
    pattern = r'R\d{5}'
    matches = re.findall(pattern, content)
    return matches

def extract_text_between_tags(html_string):
    pattern = r'ghi:\d{9}'
    matches = re.findall(pattern, html_string)
    return matches[0]

def check_kegg_ontology_id(input_string):
    pattern = r'^K\d{5} '
    if re.match(pattern, input_string):
        return True
    else:
        return False

response = requests.get('https://www.genome.jp/dbget-bin/get_linkdb?-t+genes+gn:T04793')

ko_reactions_mapping = {}
for line in response.text.split('\n'):
    if line.startswith('<a href="/entry'):
        elements = line.split('        ')
        ncbi_gene_id = extract_text_between_tags(elements[0])
        if check_kegg_ontology_id(elements[1]):
            ko_id = elements[1].split(' ')[0]
            content = query_kegg_ontology(ko_id)
            reactions = get_kegg_reaction_id(content)
            ko_reactions_mapping[ko_id] = reactions

ko_reactions_mapping

{'K00031': ['R00267',
  'R00268',
  'R01899',
  'R02360',
  'R38720',
  'R39310',
  'R39330',
  'R14101',
  'R50345',
  'R50912',
  'R70723',
  'R70331'],
 'K25841': [],
 'K12881': [],
 'K11254': ['R06800'],
 'K08678': ['R01384', 'R14101', 'R70723'],
 'K10575': [],
 'K03217': ['R42300',
  'R60400',
  'R14101',
  'R50345',
  'R50912',
  'R50912',
  'R70723',
  'R70723',
  'R70331',
  'R70331',
  'R20291',
  'R20291',
  'R50277',
  'R50277'],
 'K03283': ['R08090'],
 'K14569': [],
 'K01537': ['R50345',
  'R50345',
  'R50345',
  'R50912',
  'R50912',
  'R70723',
  'R70723',
  'R70331',
  'R70331',
  'R20291',
  'R20291',
  'R20291',
  'R20291'],
 'K13993': ['R50345',
  'R50912',
  'R70723',
  'R70331',
  'R50277',
  'R50277',
  'R50277'],
 'K11671': [],
 'K12450': ['R00293'],
 'K10398': ['R06540'],
 'K03018': [],
 'K20715': [],
 'K04125': ['R03008',
  'R03809',
  'R06337',
  'R06338',
  'R06342',
  'R06343',
  'R06344',
  'R06345'],
 'K15382': [],
 'K17943': [],
 'K11982': [],
 'K16283': [

# End of model reconstruction notebook. 

**Make sure to copy your functional model over to the results app to analyze its results in a browser**